In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
# df = pd.read_excel("imdb-dataset.xlsx")
df = pd.read_excel("imdb-dataset2.xlsx")
df.head()

,Series name,URL,Language 1,Language 2,Language 3,Language 4,Series Actor 1,Series Actor 2,Series Actor 3,Series Director 1,...,Series Writer 5,Series Writer 6,Series Writer 7,Series Executive Producer 1,Series Executive Producer 2,Series Executive Producer 3,Series Line Producer 1,Series Line Producer 2,Series Line Producer 3,Plot summary
0,House of Cards,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Hemlock Grove,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Orange Is the New Black,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Marco Polo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bloodline,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
def get_basic_details(url = "https://www.imdb.com/title/tt6473300/"):
    res = requests.get(url)
    html_page = res.content
    soup = BeautifulSoup(html_page, "html.parser")
    
    #Done - name, language, plot, story

    name = soup.h1.text.strip()
    language = soup.find_all("div", {"class": "txt-block"})

    for item in language:
        if "Language:" in (item.text):
            language = item.text.split(":")[1].strip()

    plot = (soup.find_all("div", {"class": "summary_text"})[0].text).strip()

    story = (soup.find_all("div", {"class": "inline canwrap"})[0].text).strip()

    if(type(name) != str):
        name = "NA"
    if(type(language) != str):
        language = "NA"
    if(type(plot) != str):
        plot = "NA"
    if(type(story) != str):
        story = "NA"
    return {"name":name, "language":language, "plot":plot, "story":story}
# name
# language
# plot
# story

In [10]:
# Cast
#Done - actors(list), directors(list), writers(list)
def get_actors_directors_writers(url = "https://www.imdb.com/title/tt6473300/"):
    res = requests.get(url+"fullcredits#cast")
    html_page = res.content
    soup = BeautifulSoup(html_page, "html.parser")
    actors_content = soup.find_all("table", {"class": "cast_list"})
    actors_content = actors_content[0].text.split("\n\n")
    actors = []
    for item in actors_content[1:]:
        if len(item.strip()) > 3 and "episode" not in item:
            actors.append(item.strip())

    directors = []
    writers = []

    cast_content = soup.find_all("table", {"class": "simpleTable simpleCreditsTable"})
    directors_content = cast_content[0].text.split("\n")
    directors = [element.strip() for element in directors_content 
                         if len(element.strip())>3 and "episode" not in element]

    writers_content = cast_content[1].text.split("\n")
    writers = [element.strip() for element in writers_content 
                       if len(element.strip())>3 and "episode" not in element]
    if(len(actors)==2):
        actors.append("NA")
    elif(len(actors)==1):
        actors.append("NA")
        actors.append("NA")
    elif(len(actors)==0):
        actors.append("NA")
        actors.append("NA")
        actors.append("NA")
        
    if(len(directors)==2):
        directors.append("NA")
    elif(len(directors)==1):
        directors.append("NA")
        directors.append("NA")
    elif(len(directors)==1):
        directors.append("NA")
        directors.append("NA")
        directors.append("NA")
        
    if(len(writers)==2):
        writers.append("NA")
    elif(len(writers)==1):
        writers.append("NA")
        writers.append("NA")
    elif(len(writers)==1):
        writers.append("NA")
        writers.append("NA")
        writers.append("NA")

    return {"actor1":actors[0], "actor2":actors[1], "actor3":actors[2], 
            "director1":directors[0], "director2":directors[1], "director3":directors[2],
            "writer1":writers[0], "writer2":writers[1], "writer3":writers[2]
           }
# actors
# directors
# writers

In [8]:
def get_producers(url = "https://www.imdb.com/title/tt6473300/"):
    res = requests.get(url+"fullcredits#cast")
    html_page = res.content
    soup = BeautifulSoup(html_page, "html.parser")
    cast_content = soup.find_all("table", {"class": "simpleTable simpleCreditsTable"})
    
    producers_content = cast_content[2].text.split("\n")
    producers_content = [element.strip() for element in producers_content if len(element.strip())>3]
    # producers = [element.strip() for element in producers_content 
    #                    if len(element.strip())>3 and "episode" not in element]
    producer_name = producers_content[0::2]
    producer_type = producers_content[1::2]

    exec_producer = []
    line_producer = []
    co_producer = []
    associate_producer = []

    for indx in range(len(producer_type)):
        if "exec" in producer_type[indx]:
            exec_producer.append(producer_name[indx])
        elif "line" in producer_type[indx]:
            line_producer.append(producer_name[indx])
        elif "co-" in producer_type[indx]:
            co_producer.append(producer_name[indx])
        elif "associate" in producer_type[indx]:
            associate_producer.append(producer_name[indx])

    if(len(exec_producer)==2):
        exec_producer.append("NA")
    elif(len(exec_producer)==1):
        exec_producer.append("NA")
        exec_producer.append("NA")
    elif(len(exec_producer)==0):
        exec_producer.append("NA")
        exec_producer.append("NA")
        exec_producer.append("NA")
    
    if(len(line_producer)==2):
        line_producer.append("NA")
    elif(len(line_producer)==1):
        line_producer.append("NA")
        line_producer.append("NA")
    elif(len(line_producer)==0):
        line_producer.append("NA")
        line_producer.append("NA")
        line_producer.append("NA")
        
    if(len(co_producer)==2):
        co_producer.append("NA")
    elif(len(co_producer)==1):
        co_producer.append("NA")
        co_producer.append("NA")
    elif(len(co_producer)==0):
        co_producer.append("NA")
        co_producer.append("NA")
        co_producer.append("NA")
    
    if(len(associate_producer)==2):
        associate_producer.append("NA")
    elif(len(associate_producer)==1):
        associate_producer.append("NA")
        associate_producer.append("NA")
    elif(len(associate_producer)==0):
        associate_producer.append("NA")
        associate_producer.append("NA")
        associate_producer.append("NA")
    
    return {"executive_producer1":exec_producer[0], "executive_producer2":exec_producer[1], "executive_producer3":exec_producer[2], 
            "line_producer1":line_producer[0], "line_producer2":line_producer[1], "line_producer3":line_producer[2],
            "co_producer1":co_producer[0], "co_producer2":co_producer[1], "co_producer3":co_producer[2],
            "associate_producer1":associate_producer[0], "associate_producer2":associate_producer[1], "associate_producer3":associate_producer[2]
           }
# exec_producer
# line_producer
# co_producer

In [7]:
def get_IMDb_df(url = "https://www.imdb.com/title/tt6473300/"):
    details = get_basic_details(url)
    cast = get_actors_directors_writers(url)
    producers = get_producers(url)

    result = {}
    result.update(details)
    result.update(cast)
    result.update(producers)

    result_df = pd.DataFrame(result, index=[1])
    return result_df

In [4]:
url = "https://www.imdb.com/title/tt6803712/"
get_basic_details(url)

{'name': 'Dharmakshetra',
 'language': 'Hindi',
 'plot': 'After the end of 18 days war of Mahabharata, Pandavas and Kauravas reach the court of Chitragupta where they have to answer their actions of their past life. In each episode one of the ...\n                    See full summary\xa0»',
 'story': 'After the end of 18 days war of Mahabharata, Pandavas and Kauravas reach the court of Chitragupta where they have to answer their actions of their past life. In each episode one of the character takes the centre stage to answer the questions asked by Chitragupta on behalf of the others.'}

In [84]:
df["Series name"].index

RangeIndex(start=0, stop=66, step=1)

In [11]:
url = "https://www.imdb.com/find?q="
result = pd.DataFrame()
for indx in df["Series name"][0:].index:
    res = requests.get(url+df["Series name"][indx])
    html_page = res.content
    soup = BeautifulSoup(html_page, "html.parser")
    content = soup.find("div", {"class": "findSection"})
    show_url = "https://www.imdb.com" + str(content.table.a['href'])
#     print(show_url)
    sample = get_IMDb_df(show_url)
    result = result.append(sample)
result

IndexError: list index out of range

In [13]:
result["name"]

1                         House of Cards
1                          Hemlock Grove
1                Orange Is the New Black
1                             Marco Polo
1                              Bloodline
1                              Daredevil
1                                 Sense8
1                                 Narcos
1                          Jessica Jones
1                           The Get Down
1                              Luke Cage
1                                 The OA
1         A Series of Unfortunate Events
1                              Iron Fist
1                         13 Reasons Why
1                              Free Rein
1                                  Gypsy
1                             Mindhunter
1                     Greenhouse Academy
1                           The Punisher
1                         Altered Carbon
1                          Seven Seconds
1                       Ponysitters Club
1                   The Innocents (1961)
1               

In [1]:
show_url = ["https://www.imdb.com/title/tt4687882/"]


In [12]:
result = pd.DataFrame()
for url in show_url:
    sample = get_IMDb_df(url)
    result = result.append(sample)
result.to_csv('patriot.csv')

In [94]:
df1 = pd.read_csv("output/imdb_output1.csv")
df2 = pd.read_csv("output/imdb_output2.csv")
df3 = pd.read_csv("output/imdb_output3.csv")
df4 = pd.read_csv("output/imdb_output4.csv")

result = pd.DataFrame()
result = result.append(df1)
result = result.append(df2)
result = result.append(df3)
result = result.append(df4)

result.to_csv('final_output.csv')

In [ ]:
result = pd.read_csv("final_output.csv")
